# Trip Distribution Class

# Data
## The result from trip generation (trip production and attraction) will be passed to trip distribution

In [102]:

from IPython.display import HTML, display
import math as math


class TripDistribution:

    def __init__(self, productions, attractions, travelTime, fare, income):
        self.productions = productions
        self.attractions = attractions
        self.travelTime = travelTime
        self.fare = fare
        self.income = income
        self.row = len(productions)
        self.col = len(attractions)
        self.possibleError = sum(productions) * 0.2
        self.error = 0

    def getGeneralizedCost(self, cost):
        return 1.0 / (cost * cost)
    
    def computeCost(self, travelTime, fare, income):
        costMatrix = [[1 for x in range(self.row)] for y in range(self.col)]
        for x in range(self.row):
            for y in range(self.col):
                costMatrix[x][y] = round(travelTime[x][y] * income[x] + fare[x][y], 2)
        print("costs")
        print(costMatrix)
        return costMatrix

    def getTripDistribution(self):
        distributions = [[self.attractions[y] for x in range(self.row)] for y in range(self.col)]
        finalDistributions = [[self.attractions[y] for x in range(self.row)] for y in range(self.col)]
        #costMatrix = [[1 for x in range(self.row)] for y in range(self.col)]
        costMatrix = self.computeCost(self.travelTime, self.fare, self.income)
        A = [1 for x in range(self.row)]
        B = [1 for x in range(self.col)]
        A = self.computeA(B, costMatrix)
        B = self.computeB(A, costMatrix)
        
        currentBalancingFactor = 0  # 0 for A, 1 for B
        isConvergent = False
        shit =0
        smallestError = 1000000000

#         while isConvergent == False:
        for x in range(10000):
            if currentBalancingFactor == 0:
                tempA = self.computeA(B, costMatrix)
                A = tempA
                currentBalancingFactor = 1
            elif currentBalancingFactor == 1:
                tempB = self.computeB(A, costMatrix)
                B = tempB
                currentBalancingFactor = 0
            distributions = self.computeDistributions(A, B, costMatrix)
            error = self.getError(distributions)
            if(smallestError > error and error != 0):
                print(error)
                print(distributions)
                smallestError = error
                finalDistributions = distributions
                self.error = error
                shit = x
#             isConvergent = self.checkIfConvergent(distributions)
        print(shit)
        return finalDistributions

    def computeDistributions(self, A, B, costMatrix):
        distributions = [[self.attractions[y] for x in range(self.row)] for y in range(self.col)]
        for x in range(self.row):
            for y in range(self.col):
                distributions[x][y] = round(A[x] * self.productions[x] * B[y] * self.attractions[y] * self.getGeneralizedCost(costMatrix[x][y]), 2)
        return distributions

    def checkIfConvergent(self, distributions):
        error = self.getError(distributions)
        if error <= self.possibleError:
            self.error = error
            return True
        return False

    def getError(self, distributions):
        error = 0
        derivedProductions = [0 for x in range(self.row)]
        derivedAttractions = [0 for x in range(self.col)]

        for x in range(self.row):
            for y in range(self.col):
                derivedProductions[x] += distributions[x][y]
                derivedAttractions[y] += distributions[x][y]

        for x in range(self.row):
            error += abs(derivedProductions[x] - self.productions[x])
            error += abs(derivedAttractions[x] - self.attractions[x])

        return error
    
    
    def getErrorPercentage(self):
        return self.error/sum(self.productions)

    def computeA(self, B, costMatrix):
        A = [1 for x in range(self.row)]
        for x in range(0, self.row):
            sum = 0.0
            for y in range(0, self.col):
                sum += B[y] * self.attractions[y] * self.getGeneralizedCost(costMatrix[x][y])
            A[x] = 1.0 / sum
        return A

    def computeB(self, A, costMatrix):
        B = [1 for x in range(self.col)]
        for x in range(0, self.row):
            sum = 0.0
            for y in range(0, self.col):
                sum += A[y] * self.productions[y] * self.getGeneralizedCost(costMatrix[x][y])
            B[x] = 1.0 / sum
        return B

In [103]:
def computeYearlyToHourlyRate(salary):
    return salary/(30.00 * 8)

def deg2rad(deg):
    return deg * (math.pi/180)

def getDistance(lat1, lng1, lat2, lng2):
    dlon = lng2 - lng1
    dlat = lat2 - lat1
    a = ((math.sin(deg2rad(dlat/2)))*(math.sin(deg2rad(dlat/2)))) + math.cos(deg2rad(lat1)) *math.cos(deg2rad(lat2)) *((math.sin(deg2rad(dlon/2)))*(math.sin(deg2rad(dlon/2))))
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d =  6373* c
    return d



def computeNearestZones(lat, lng) :
    nearestZones = [None] * len(lat)
    for currZone in range(len(lat)):
        shortest = 1000
        nearestZoneIndex = 0
        for currPoint in range(len(lat)):
            if(currZone != currPoint): 
                distance = getDistance(lat[currZone], lng[currZone], lat[currPoint], lng[currPoint])
                if(distance < shortest):
                    shortest = distance
                    nearestZoneIndex = currPoint
        nearestZones[currZone] = nearestZoneIndex
        shortest = 1000
        nearestZoneIndex = 0
    return nearestZones

productions = [
    102989, 
    73253,
    92057,
    15821,
    56998, 
    103039, 
    32610, 
    172844
]
attractions = [
    79305.39, 
    81250.91, 
    76218.9, 
    11167.77, 
    38786.66, 
    133188.75, 
    50085.41, 
    179607.21
]

nearestZones = [
    0, 
    0,
    0,
    0,
    0,
    0,
    0,
    0
]

lat = [
    14.711029630751996,
    14.681442858628275,
    14.708693067291241, 
    14.686174778272566, 
    14.660783607519472,
    14.65615255044289,
    14.630416868404804,
    14.637488044822764
]

lng = [
    120.960596201819,
    120.98000062117379,
    121.00151327979542,
    121.00714359391715,
    121.09957600978247,
    121.11920195444038,
    121.080185938878,
    121.09795399889786
]

travelTime = [
        [001.00, 027.00, 053.67, 056.67, 140.67, 142.67, 114.67, 123.67], 
        [028.33, 001.00, 050.67, 051.00, 124.33, 132.00, 098.33, 106.67],
        [052.33, 050.33, 001.00, 037.00, 127.67, 143.00, 109.00, 117.33], 
        [057.00, 051.00, 037.00, 001.00, 122.67, 130.00, 096.33, 104.67],
        [151.33, 133.67, 136.00, 131.33, 001.00, 031.67, 038.67, 026.67],
        [157.00, 139.33, 149.67, 137.33, 032.00, 001.00, 041.67, 030.33],
        [120.00, 102.00, 113.00, 096.67, 037.00, 040.67, 001.00, 015.00], 
        [129.00, 111.00, 121.67, 105.33, 029.00, 029.67, 015.00, 001.00]
        ]

fares = [
        [01.00, 10.67, 17.65, 25.33, 54.35, 53.55, 47.00, 48.65], 
        [12.00, 01.00, 56.65, 16.37, 45.60, 45.14, 38.58, 40.23],
        [17.65, 16.22, 01.00, 10.67, 43.72, 46.08, 40.25, 40.57], 
        [16.67, 18.67, 10.67, 01.00, 42.02, 41.64, 35.08, 36.73],
        [52.00, 43.33, 40.97, 39.41, 01.00, 10.67, 11.31, 08.00], 
        [52.77, 44.10, 45.35, 40.27, 08.00, 01.00, 10.64, 08.03],
        [45.17, 36.50, 37.83, 34.50, 11.01, 10.24, 01.00, 08.00], 
        [47.43, 38.76, 40.09, 36.03, 09.33, 08.00, 08.00, 01.00]
        ]
salary = [computeYearlyToHourlyRate(24570.49), 
          computeYearlyToHourlyRate(16576.02), 
          computeYearlyToHourlyRate(21038.92), 
          computeYearlyToHourlyRate(16985.72), 
          computeYearlyToHourlyRate(29038.00), 
          computeYearlyToHourlyRate(30548.05), 
          computeYearlyToHourlyRate(27276.91), 
          computeYearlyToHourlyRate(39221.55)];
            
        

    
nearestZones = computeNearestZones(lat, lng)

for x in range(len(productions)): 
    fares[x][x] = fares[x][nearestZones[x]] / 2
    travelTime[x][x] = travelTime[x][nearestZones[x]] / 2

td = TripDistribution(productions, attractions, travelTime, fares, salary)


print("fare")
print(fares)
for x in range(len(productions)):
    salary[x] = round(salary[x],2)
    for y in range(len(productions)):
        travelTime[x][y] = travelTime[x][y]/60

print("trave time")
print(travelTime)

print("salary")
print(salary)

print("Productions: " + str(productions))
print("Attractions: " + str(attractions))

fare
[[5.335, 10.67, 17.65, 25.33, 54.35, 53.55, 47.0, 48.65], [12.0, 8.185, 56.65, 16.37, 45.6, 45.14, 38.58, 40.23], [17.65, 16.22, 5.335, 10.67, 43.72, 46.08, 40.25, 40.57], [16.67, 18.67, 10.67, 5.335, 42.02, 41.64, 35.08, 36.73], [52.0, 43.33, 40.97, 39.41, 5.335, 10.67, 11.31, 8.0], [52.77, 44.1, 45.35, 40.27, 8.0, 4.0, 10.64, 8.03], [45.17, 36.5, 37.83, 34.5, 11.01, 10.24, 4.0, 8.0], [47.43, 38.76, 40.09, 36.03, 9.33, 8.0, 8.0, 4.0]]
trave time
[[0.225, 0.45, 0.8945000000000001, 0.9445, 2.3444999999999996, 2.3778333333333332, 1.9111666666666667, 2.061166666666667], [0.4721666666666666, 0.425, 0.8445, 0.85, 2.0721666666666665, 2.2, 1.6388333333333334, 1.7778333333333334], [0.8721666666666666, 0.8388333333333333, 0.30833333333333335, 0.6166666666666667, 2.1278333333333332, 2.3833333333333333, 1.8166666666666667, 1.9555], [0.95, 0.85, 0.6166666666666667, 0.30833333333333335, 2.0445, 2.1666666666666665, 1.6055, 1.7445], [2.5221666666666667, 2.2278333333333333, 2.2666666666666666, 2.

In [104]:
distribution = td.getTripDistribution()
sumProd = [None] * len(distribution)
sumAttr = [0] * len(distribution)
for x in range(len(distribution)):
    sumProd[x] = sum(distribution[x])
    for y in range(len(distribution[x])):
        sumAttr[y] += distribution[x][y]

print((sumProd))
print((sumAttr))

display(HTML(
    '<table><tr>{}</tr></table>'.format(
        '</tr><tr>'.join(
            '<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row)) for row in distribution)
        )
))

costs
[[28.37, 56.74, 109.23, 122.03, 294.38, 296.99, 242.67, 259.67], [44.61, 37.54, 114.98, 75.08, 188.72, 197.09, 151.77, 163.02], [94.1, 89.75, 32.36, 64.73, 230.25, 255.0, 199.5, 211.99], [83.9, 78.82, 54.31, 27.16, 186.71, 194.97, 148.7, 160.19], [357.16, 312.88, 315.21, 304.24, 37.27, 74.53, 89.29, 61.78], [385.82, 339.67, 362.85, 331.59, 75.88, 37.94, 99.04, 72.37], [272.47, 229.71, 251.87, 217.61, 81.09, 87.28, 18.21, 36.41], [398.78, 341.09, 371.48, 322.91, 88.32, 88.81, 48.85, 24.43]]
97752.51999999996
[[69227.04, 22526.3, 5059.25, 778.75, 573.72, 1814.39, 817.95, 2191.59], [20664.46, 37981.69, 3369.92, 1518.37, 1030.33, 3040.74, 1543.41, 4104.07], [6928.53, 9913.47, 63471.55, 3047.53, 1032.63, 2709.94, 1332.6, 3620.74], [1805.81, 2663.16, 4668.87, 3586.53, 325.38, 960.46, 496.98, 1313.81], [223.74, 379.47, 311.2, 64.18, 18334.53, 14757.76, 3094.73, 19832.39], [249.62, 419.19, 305.75, 70.34, 5758.66, 74143.95, 3274.88, 18816.61], [84.8, 155.29, 107.51, 27.67, 854.3, 2373.61,

64318.34,22780.26,5350.56,858.36,1086.36,3765.13,1336.54,3493.46
17542.38,35095.23,3256.4,1529.16,1782.6,5765.46,2304.31,5977.47
5782.78,9005.98,60301.5,3017.54,1756.52,5051.79,1956.09,5184.79
1413.61,2269.16,4160.3,3330.77,519.11,1679.3,684.21,1764.53
115.03,212.35,182.12,39.14,19210.95,16946.46,2798.24,17493.7
118.79,217.13,165.63,39.71,5585.2,78808.3,2740.91,15363.34
47.74,95.16,68.89,18.48,980.19,2984.63,16249.84,12165.07
116.4,225.4,165.41,43.83,4315.52,15055.7,11793.55,141128.18


# Error


The following formula, which is from a book, Introduction to Transportation Engineering by Tom V. Mathew and K V Krishna Rao, is used:

$$T_{ij} = A_iO_iB_jD_jf(c_{ij})$$

Where $T_{ij}$ is the trips made from zone i to the zone j, $O_i$ refers to the trip productions made from zone i, $D_j$ refers to the trip attractions made from zone j, $c_{ij}$ is the general cost used from zone i to zone j, $A_i$ and $B_i$ are the balancing factor and the following formula are used:

 $$A_i = 1 / \sum_{j} B_{j}D_jf(c_{ij})$$
 $$B_i = 1 / \sum_{i} A_{i}O_if(c_{ij})$$
 
At first, one of the balancing factors will be set to 1s and the other will be compupted using the formula above. This will be done alternately until the some of all the origins or the destinations of the computed distributions will be close to the original productions and attractions. Lastly, the function $f(c_{ij})$ has the following formula:
$$f(c_{ij}) = 1/c_{ij}^2$$

$$c_{ij} = t_{ij}s_i + f_{ij}$$



In [84]:
print("Error: " + str(td.getError(distribution)))
print(td.getErrorPercentage())

Error: 83850.12000000002
0.1290774324942158
